# Mercedes Benz

In [9]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use('ggplot')
%matplotlib inline

In [10]:
train_file = 'training/train.csv'
test_file = 'training/test.csv'

train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

In [11]:
train_df.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [12]:
test_df.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0


In [46]:
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import ModelCheckpoint
from keras_tqdm import TQDMNotebookCallback

# To evaluate models using cross-validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
# To perform data preparation in order to improve skill with Keras models
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# Ensemble
from sklearn.ensemble import RandomForestRegressor
# Feature Selection
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
import pickle

In [47]:
for c in train_df.columns:
    if train_df[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(train_df[c].values)) 
        train_df[c] = lbl.transform(list(train_df[c].values))
        
for c in test_df.columns:
    if test_df[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(test_df[c].values)) 
        test_df[c] = lbl.transform(list(test_df[c].values))

In [ ]:
# define base model
def simple_model():
    # create model
    # Topology 13 inputs -> [13 -> 6] -> 1 output
    model = Sequential()
    model.add(Dense(376, input_dim=376, kernel_initializer='normal', activation='relu'))
    model.add(Dense(36, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

id_test = test_df['ID']
test_noID = test_df.drop("ID", axis=1).copy()

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# model = simple_model()
estimator = KerasRegressor(build_fn=simple_model, epochs=500, batch_size=5, verbose=1)


# Numpy representation of NDFrame
dataset = train_df.values
# split into input (X) and output (Y) variables
X = dataset[:,2:378]
Y = dataset[:,1]

datasetTest = test_noID.values
# split into input (X) and output (Y) variables
X_test = datasetTest[:,0:376]

trainData, testData, trainLabels, testLabels = train_test_split(X, Y, random_state=20)

filepath = "simple.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# model.fit(X, Y,
#           epochs=500,
#           batch_size=5,
#           verbose=0,
#           callbacks=[TQDMNotebookCallback()])

estimator.fit(X,Y)
res = estimator.predict(X_test)

estimator.model.save('simple.h5')

Epoch 1/500
4209/4209 [==============================] - 3s - loss: 738.4870     
Epoch 2/500
4209/4209 [==============================] - 2s - loss: 88.0662     
Epoch 3/500
4209/4209 [==============================] - 2s - loss: 83.6828     
Epoch 4/500
4209/4209 [==============================] - 2s - loss: 81.8894     
Epoch 5/500
4209/4209 [==============================] - 2s - loss: 82.2909     
Epoch 6/500
4209/4209 [==============================] - 2s - loss: 79.1451     
Epoch 7/500
4209/4209 [==============================] - 2s - loss: 78.1143     
Epoch 8/500
4209/4209 [==============================] - 2s - loss: 78.2467     
Epoch 9/500
4209/4209 [==============================] - 2s - loss: 77.3024     
Epoch 10/500
4209/4209 [==============================] - 2s - loss: 74.4158     
Epoch 11/500
4209/4209 [==============================] - 2s - loss: 75.0738     
Epoch 12/500
4209/4209 [==============================] - 2s - loss: 76.3576     
Epoch 13/500
4209/4209 [

In [51]:
from keras.models import load_model

In [52]:
# model = load_model('simple.h5')
res = estimator.predict(X_test)
output = pd.DataFrame({'id': id_test, 'y': res})
output.to_csv('preds.csv', index=False)

Exception: Data must be 1-dimensional

In [31]:
output.head()

89.0